In [2]:
from __future__ import print_function
import cv2
import numpy as np


MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15

In [3]:
def alignImages(im1, im2):

  # Convert images to grayscale
  im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
  im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
  
  # Detect ORB features and compute descriptors.
  orb = cv2.ORB_create(MAX_FEATURES)
  keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
  keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
  
  # Match features.
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
  matches = matcher.match(descriptors1, descriptors2, None)
  
  # Sort matches by score
  matches.sort(key=lambda x: x.distance, reverse=False)

  # Remove not so good matches
  numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
  matches = matches[:numGoodMatches]

  # Draw top matches
  imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
  cv2.imwrite("matches.jpg", imMatches)
  
  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros((len(matches), 2), dtype=np.float32)

  for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt
  
  # Find homography
  h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

  # Use homography
  height, width, channels = im2.shape
  im1Reg = cv2.warpPerspective(im1, h, (width, height))
  
  return im1Reg, h

In [4]:
import os, glob
# put all images in the same folder (reflectance panel and the rest)
path_inputs = (r'C:\Users\Alfonso\Desktop\20200727_1150AM_Matrice_Micasense_IRT_Tower1\0000SET\thumbnails_selected')   
path_outputs = (r'C:\Users\Alfonso\Desktop\20200727_1150AM_Matrice_Micasense_IRT_Tower1\0000SET\thumbnails_aligned')

# Read reference image
refFilename = os.path.join(path_inputs,'Capture19_03_32.1.jpg')
print("Reading reference image : ", refFilename)
imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)

imReference = cv2.rotate(imReference, cv2.ROTATE_90_COUNTERCLOCKWISE) 

Reading reference image :  C:\Users\Alfonso\Desktop\20200727_1150AM_Matrice_Micasense_IRT_Tower1\0000SET\thumbnails_selected\Capture19_03_32.1.jpg


In [5]:
files = glob.glob(path_inputs+'/*.jpg')

for file in files:

    # Read image to be aligned
    # imFilename = "Capture19_03_35.1.jpg"
    print("Reading image to align : ", file);  
    im = cv2.imread(file, cv2.IMREAD_COLOR)
    im = cv2.rotate(im, cv2.ROTATE_90_COUNTERCLOCKWISE) 
    filename = os.path.split(file)

    print("Aligning images ...")
    # Registered image will be resotred in imReg. 
    # The estimated homography will be stored in h. 
    imReg, h = alignImages(im, imReference)

    # Write aligned image to disk. 
    outFilename = filename[-1]
    print("Saving aligned image : ", outFilename); 
    cv2.imwrite(os.path.join(path_outputs,outFilename), imReg)

    # Print estimated homography
#     print("Estimated homography : \n",  h)
  

Reading image to align :  C:\Users\Alfonso\Desktop\20200727_1150AM_Matrice_Micasense_IRT_Tower1\0000SET\thumbnails_selected\Capture18_45_53.1.jpg
Aligning images ...
Saving aligned image :  Capture18_45_53.1.jpg
Reading image to align :  C:\Users\Alfonso\Desktop\20200727_1150AM_Matrice_Micasense_IRT_Tower1\0000SET\thumbnails_selected\Capture18_45_56.1.jpg
Aligning images ...
Saving aligned image :  Capture18_45_56.1.jpg
Reading image to align :  C:\Users\Alfonso\Desktop\20200727_1150AM_Matrice_Micasense_IRT_Tower1\0000SET\thumbnails_selected\Capture18_45_57.1.jpg
Aligning images ...
Saving aligned image :  Capture18_45_57.1.jpg
Reading image to align :  C:\Users\Alfonso\Desktop\20200727_1150AM_Matrice_Micasense_IRT_Tower1\0000SET\thumbnails_selected\Capture18_45_58.1.jpg
Aligning images ...
Saving aligned image :  Capture18_45_58.1.jpg
Reading image to align :  C:\Users\Alfonso\Desktop\20200727_1150AM_Matrice_Micasense_IRT_Tower1\0000SET\thumbnails_selected\Capture18_45_59.1.jpg
Aligni